# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686496


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:04<01:01,  2.21s/it]

Rendering models:  10%|█         | 3/30 [00:07<01:04,  2.38s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:46,  1.81s/it]

Rendering models:  17%|█▋        | 5/30 [00:08<00:34,  1.40s/it]

...

Rendering models:  87%|████████▋ | 26/30 [00:12<00:00,  4.08it/s]

Rendering models:  90%|█████████ | 27/30 [00:12<00:00,  3.12it/s]

Rendering models:  93%|█████████▎| 28/30 [00:12<00:00,  3.25it/s]

Rendering models:  97%|█████████▋| 29/30 [00:12<00:00,  3.98it/s]

not-logged-in-20a275b53abec8db9f41    0.006756
not-logged-in-bc5fdb9ea8d611e60b8b    0.000442
not-logged-in-5fb8d79457d3ba01439f    0.000492
asucich                               0.000532
djswanso                              0.000461
not-logged-in-43b69edd3353a5d3fc17    0.000910
kjkavene                              0.000467
Alexatwin                             0.003487
not-logged-in-5f30b2bb9b7f9d49da41    0.003624
ljimenez23                            0.009274
pangeli5                              0.000652
Lavadude                              0.045680
hpatss1                               0.000467
jmacd297                              0.002003
not-logged-in-242e8a513bddb973a39c    0.001119
not-logged-in-490f77176b83d659546b    0.002748
ElisabethB                            0.026196
ChristanM                             0.001513
Octavius_Augustus                     0.000846
not-logged-in-5c973654fb0968b87ca1    0.002908
BeatrizAltern                         0.003727
LAMBO101799  

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())